In [2]:
import networkx as nx #https://networkx.github.io/
import json
import community 
from IPython.display import HTML

outputFile = "graph.json"
f = open(outputFile, 'wb')
graphsize = 100
nodes = {}
links = {}
Graph = nx.barabasi_albert_graph(graphsize, 1)
partition = community.best_partition(Graph)
dc = nx.degree_centrality(Graph)

j = "" # JSON object
j = j + "{"    
j = j + """\t"nodes": ["""

for n in nx.nodes(Graph):
    nodes[n] = {}
    nodes[n]['name'] = n
    nodes[n]['group'] = partition[n]
    nodes[n]['dc'] = dc[n]
for n in nodes:
    j = j + str(json.dumps(nodes[n])) + ",\n"
j = j[:-2]
j = j + "\t],\n"
j = j + """\t"links":[\n"""
for link in nx.edges(Graph):
    links[str(link)] = {}
    links[str(link)]['source'] = link[0]
    links[str(link)]['target'] = link[1]
for l in links:
    j = j + str(json.dumps(links[l])) + ",\n"
j = j[:-2]
j = j + "\t]\n"
j = j + "}"
f.write(j)
f.close()

In [10]:
%%javascript
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// create the network (nodes and links)
require(["d3"], function(d3) {
    window.nodes = [];
    window.links = [];
    d3.json("miserables.json", function(error, graph) {
        console.log(graph);
        if (error) throw error;

        graph.nodes.forEach(function(d,i) {
            nodes.push({
                name:d.name,
                group:d.group,
                dc:d.dc,
                index: i,
                //fx:i*10,
                //fy:i*10,
            })
        })
        graph.links.forEach(function(d,i) {
            links.push({
                source:nodes[d.source],
                target:nodes[d.target],
                index: i,
                value:d.value,
            })
        })
    })
})

<IPython.core.display.Javascript object>

In [11]:
%%javascript
element.append('<div id="graph1" style="min-width: 310px; height: 1000px; margin: 0 auto"></div>');
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// Original Example: https://bl.ocks.org/mbostock/4062045
require(["d3"], function(d3) {
    // Use this to keep the screen clean
    d3.select("div#graph1").selectAll("*").remove();    
    
    var width = 800, height = 1000;
    // Create SVG
    var svg = d3.select("div#graph1").append("svg")
        .attr("width", (width)+"px")
        .attr("height", (height)+"px")
    // create color schema 
    var color = d3.scaleOrdinal(d3.schemeCategory20); // https://github.com/d3/d3-scale

    var simulation = d3.forceSimulation() // https://github.com/d3/d3-force
        .force("link", d3.forceLink().id(function(d) { return d.index; }))
        .force("charge", d3.forceManyBody())
        .force("center", d3.forceCenter(width / 2, height / 2));

    var link = svg.append("g") // https://www.w3schools.com/graphics/svg_line.asp
        .attr("class", "links")
        .selectAll("line").data(links).enter().append("line")
        .style("stroke", "black")
        .style("stroke-width", function(d) { return "2px"; });

    var node = svg.append("g") // https://www.w3schools.com/graphics/svg_circle.asp
        .attr("class", "nodes")
        .selectAll("circle")
        .data(nodes)
        .enter().append("circle")
        .attr("r", 5)
        .attr("fill", function(d) { return color(d.group); })

    node.append("title")
        .text(function(d) { return d.name; });

    simulation
        .nodes(nodes)
        .on("tick", ticked);

    simulation.force("link")
        .links(links);

    function ticked() {
        link
            .attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        node
            .attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
    }


})

<IPython.core.display.Javascript object>

In [12]:
%%javascript
element.append('<div id="graph2" style="min-width: 310px; height: 1000px; margin: 0 auto"></div>');
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// Original Example: https://bl.ocks.org/mbostock/4062045
require(["d3"], function(d3) {
    // Use this to keep the screen clean
    d3.select("div#graph2").selectAll("*").remove();    
    
    var width = 800, height = 1000;
    var svg = d3.select("div#graph2").append("svg")
        .attr("width", (width)+"px")
        .attr("height", (height)+"px")
    var color = d3.scaleOrdinal(d3.schemeCategory20);

    var link = svg.append("g")
        .attr("class", "links")
        .selectAll("line").data(links).enter().append("line")
        .style("stroke", "black")
        .style("stroke-width", function(d) { return "2px"; });

    var node = svg.append("g")
        .attr("class", "nodes")
        .selectAll("circle")
        .data(nodes)
        .enter().append("circle")
        .attr("r", 5)
        .style("fill", function(d) { return color(d.group); })

    node.append("title")
        .text(function(d) { return d.name; });

    var simulation = d3.forceSimulation()
        .force("link", d3.forceLink().id(function(d) { return d.index; })
               .distance(function(d){ return (d.source.group==d.target.group) ? 30 : 30 })
               .strength(1)) // link distance/strength 
        .force("x", d3.forceX(function(d) { return d.x; }).strength(0))
        .force("y", d3.forceY(function(d) { return d.y; }).strength(0))
        .force("charge", d3.forceManyBody().strength([-30]) ) //many-body force (positive=gravity|negative=electrostatic charge)
        .force("collide", d3.forceCollide(0)) // node repulsion
        .force("center", d3.forceCenter(width / 2, height / 2)); // where the center of the graph is located
    
    simulation
        .nodes(nodes)
        .on("tick", ticked);

    simulation.force("link")
        .links(links);

    function ticked() {
        link
            .attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        node
            .attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
    }


})

<IPython.core.display.Javascript object>

In [16]:
%%javascript
element.append('<div id="graph3" style="min-width: 310px; height: 1000px; margin: 0 auto"></div>');
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// Original Example: https://bl.ocks.org/mbostock/4062045
require(["d3"], function(d3) {
    // Use this to keep the screen clean
    d3.select("div#graph3").selectAll("*").remove();    
    
    var width = 800, height = 1000;
    var svg = d3.select("div#graph3").append("svg")
        .attr("width", (width)+"px")
        .attr("height", (height)+"px")
    var color = d3.scaleOrdinal(d3.schemeCategory20);

    var link = svg.append("g")
        .attr("class", "links")
        .selectAll("line").data(links).enter().append("line")
        .style("stroke", "black")
        .style("stroke-width", function(d) { return d.value; });

    var node = svg.append("g")
        .attr("class", "nodes")
        .selectAll("circle")
        .data(nodes)
        .enter().append("circle")
        .attr("r", function(d) { return 5; })
        .style("fill", function(d) { return color(d.group); })

    node.append("title")
        .text(function(d) { return d.name; });

    var simulation = d3.forceSimulation()
        .force("link", d3.forceLink().id(function(d) { return d.index; })
               .distance(function(d){ return (d.source.group==d.target.group) ? 10 : 100 })
               .strength(1)) // link distance/strength 
        .force("x", d3.forceX(function(d) { return d.x; }).strength(0))
        .force("y", d3.forceY(function(d) { return d.y; }).strength(0))
        .force("charge", d3.forceManyBody().strength([-30]) ) //many-body force (positive=gravity|negative=electrostatic charge)
        .force("collide", d3.forceCollide(0)) // node repulsion
        .force("center", d3.forceCenter(width / 2, height / 2)); // where the center of the graph is located
    
    simulation
        .nodes(nodes)
        .on("tick", ticked);

    simulation.force("link")
        .links(links);

    function ticked() {
        link
            .attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        node
            .attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
    }


})

<IPython.core.display.Javascript object>

In [7]:
%%javascript
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// This adds drag functionality
// https://github.com/d3/d3-drag
require(["d3"], function(d3) {
window.dragstarted = function(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

window.dragged = function(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

window.dragended = function(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}
})

<IPython.core.display.Javascript object>

In [15]:
%%javascript
element.append('<div id="graph4" style="min-width: 310px; height: 1000px; margin: 0 auto"></div>');
require.config({paths: {d3: "//d3js.org/d3.v4.min",}}); // Home directory of D3.JS: https://d3js.org/
// Original Example: https://bl.ocks.org/mbostock/4062045
require(["d3"], function(d3) {
    // Use this to keep the screen clean
    d3.select("div#graph4").selectAll("*").remove();    
    
    var width = 800, height = 1000;
    var svg = d3.select("div#graph4").append("svg")
        .attr("width", (width)+"px")
        .attr("height", (height)+"px")
    var color = d3.scaleOrdinal(d3.schemeCategory20);
    
    window.simulation = d3.forceSimulation()
        .force("link", d3.forceLink().id(function(d) { return d.index; }))
        .force("charge", d3.forceManyBody()) //many-body force (positive=gravity|negative=electrostatic charge)
        .force("collide", d3.forceCollide(5)) // node repulsion
        .force("center", d3.forceCenter(width / 2, height / 2)); // where the center of the graph is located
    
    var link = svg.append("g")
        .attr("class", "links")
        .selectAll("line").data(links).enter().append("line")
        .style("stroke", "black")
        .style("stroke-width", function(d) { return "2px"; });

    var node = svg.append("g")
        .attr("class", "nodes")
        .selectAll("circle")
        .data(nodes)
        .enter().append("circle")
        .attr("r", function(d) { return 5; })
        .style("fill", function(d) { return color(d.group); })
        .call(d3.drag()
            .on("start", dragstarted)
            .on("drag", dragged)
            .on("end", dragended));

    node.append("title")
        .text(function(d) { return d.name; });

    simulation
        .nodes(nodes)
        .on("tick", ticked);

    simulation.force("link")
        .links(links);

    function ticked() {
        link
            .attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });

        node
            .attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
    }


})

<IPython.core.display.Javascript object>